In [39]:
using Pkg
Pkg.activate(".")
Pkg.add("PowerModelsDistribution")
Pkg.add("Ipopt")
Pkg.add("JuMP")
Pkg.add("HiGHS")

  Activating environment at `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Project.toml`


   Resolving package versions...


  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Project.toml`
  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Manifest.toml`


   Resolving package versions...


  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Project.toml`
  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Manifest.toml`


   Resolving package versions...


  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Project.toml`
  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Manifest.toml`


   Resolving package versions...


  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Project.toml`
  No Changes to `~/dev/Julia/DynaGrid/OnlineOptDynaGrid/Manifest.toml`


In [40]:
using PowerModelsDistribution
const PMD = PowerModelsDistribution
using OnlineOptDynaGrid
import Ipopt
import JuMP
import HiGHS

In [51]:
pmd_path = joinpath(dirname(pathof(PowerModelsDistribution)), "..")
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
HiGHS_solver = optimizer_with_attributes(
	HiGHS.Optimizer,
	"presolve"=>"off",
	"primal_feasibility_tolerance"=>1e-6,
	"dual_feasibility_tolerance"=>1e-6,
	"mip_feasibility_tolerance"=>1e-4,
	"mip_rel_gap"=>1e-4,
	"small_matrix_value"=>1e-12,
	"allow_unbounded_or_infeasible"=>true
)

# case_file = joinpath(pmd_path, "test/data/opendss/IEEE13_Assets.dss")
case_file = "./data/dss/IEEE_13_new.dss"


case = parse_file(case_file)

case_math = transform_data_model(case)

pm = instantiate_mc_model(case_math, ACRUPowerModel, build_mc_opf);
var(pm, 0)

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'IEEE_13_new.dss'
┌ PowerModelsDistribution | Warning ] : Command 'calcv' on line 167 in 'IEEE_13_new.dss' is not recognized, skipping.
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/4Twb2/src/io/dss/dss_parse.jl:967
[ PowerModelsDistribution | Info ] : Command 'solve' on line 176 in 'IEEE_13_new.dss' is not supported, skipping.


Dict{Symbol, Any} with 22 entries:
  :p      => Dict{Tuple{Int64, Int64, Int64}, JuMP.Containers.DenseAxisArray{Ju…
  :vi     => Dict{Int64, JuMP.Containers.DenseAxisArray{JuMP.VariableRef, 1, Tu…
  :se     => 1-dimensional DenseAxisArray{JuMP.VariableRef,1,...} with index se…
  :qsw    => Dict{Any, Any}((3, 11, 23)=>1-dimensional DenseAxisArray{JuMP.Vari…
  :ps     => Dict{Int64, JuMP.Containers.DenseAxisArray{JuMP.VariableRef, 1, Tu…
  :qs     => Dict{Int64, JuMP.Containers.DenseAxisArray{JuMP.VariableRef, 1, Tu…
  :qg_bus => Dict{Int64, Any}(4=>1-dimensional DenseAxisArray{JuMP.VariableRef,…
  :sc     => 1-dimensional DenseAxisArray{JuMP.VariableRef,1,...} with index se…
  :qg     => Dict{Int64, JuMP.Containers.DenseAxisArray{JuMP.VariableRef, 1, Tu…
  :qsc    => 1-dimensional DenseAxisArray{JuMP.VariableRef,1,...} with index se…
  :pt     => Dict{Tuple{Int64, Int64, Int64}, JuMP.Containers.DenseAxisArray{Ju…
  :qd     => Dict{Int64, Any}()
  :pg_bus => Dict{Int64, Any}(4=>1-dimensi

In [42]:
# Solve the power flow equations using Ipopt
# Power flow model is ACRUPowerModel: AC Rectangular Unbalanced
result = solve_mc_pf(case, ACRUPowerModel, ipopt_solver)

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.325864
  "optimizer"          => "Ipopt"
  "termination_status" => ALMOST_LOCALLY_SOLVED
  "dual_status"        => NEARLY_FEASIBLE_POINT
  "primal_status"      => NEARLY_FEASIBLE_POINT
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("bus"=>Dict{String, Any}("800aux"=>…
  "objective_lb"       => -Inf